In [ ]:
import sys
sys.path.append("../")  

In [22]:
import torch
import torch.optim as optim
from torch import nn

import hgp
from hgp.models import HGNNP,CHGNN
from hgp.function import StraightThroughEstimator

import numpy as np
import random
DEVICE = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
DEVICE


device(type='cuda', index=0)

In [ ]:
seed = 42
torch.manual_seed(seed) 
torch.cuda.manual_seed(seed) 
torch.cuda.manual_seed_all(seed)  
np.random.seed(seed) 
random.seed(seed)  
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [24]:
from hgp.models import ParameterDict

# fmt: off
h_hyper_prmts = ParameterDict()
l_hyper_prmts = ParameterDict()

weight = 5


partitions = 4

h_hyper_prmts["convlayers1"] = {"in_channels": 1330, "out_channels": 1024, "use_bn": False, "drop_rate": 0.2}
h_hyper_prmts["convlayers2"] = {"in_channels": 1024, "out_channels": 512, "use_bn": False, "drop_rate": 0.1}
h_hyper_prmts["convlayers3"] = {"in_channels": 512, "out_channels": 256, "use_bn": False, "drop_rate": 0.1}
h_hyper_prmts["convlayers5"] = {"in_channels": 256, "out_channels": 128, "use_bn": True, "drop_rate": 0.1}
l_hyper_prmts["linerlayer1"] = {"in_channels":list(h_hyper_prmts.values())[-1]["out_channels"], "out_channels":128, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer21"] = {"in_channels":128, "out_channels":64, "use_bn":False, "drop_rate":0.05}
l_hyper_prmts["linerlayer4"] = {"in_channels":64, "out_channels":4, "use_bn":False, "drop_rate":0.05}

# h_hyper_prmts["convlayers1"] = {"in_channels": 1330, "out_channels": 1024, "use_bn": False, "drop_rate": 0.2}
# h_hyper_prmts["convlayers2"] = {"in_channels": 1024, "out_channels": 512, "use_bn": False, "drop_rate": 0.1}
# h_hyper_prmts["convlayers3"] = {"in_channels": 512, "out_channels": 128, "use_bn": False, "drop_rate": 0.1}
# h_hyper_prmts["convlayers5"] = {"in_channels": 128, "out_channels": 4, "use_bn": False, "drop_rate": 0.1}


hyper = {
    "h_hyper_prmts": h_hyper_prmts,
    "l_hyper_prmts":l_hyper_prmts,
    "init_features_dim":list(h_hyper_prmts.values())[0]["in_channels"],
    "partitions":partitions
}

# fmt: on

In [ ]:
def loss_bs_matrix(outs, hg, device, weight):

    H = hg.H.to_dense().to(device)
    outs = outs.to(device)

    X_ = outs.t().unsqueeze(-1)
    H_ = H.unsqueeze(0)
    xweight = H.sum(dim=0)
    mid = X_.mul(H_)
    sum = (mid * (1 / xweight)).sum()
    sub = (mid + (1 - H)).prod(dim=1).sum()
    loss_1 = sum - sub

    loss_2 = torch.var(torch.sum(outs, dim=0)).to(device)
    loss = weight * loss_1 + loss_2

    return loss, loss_1, loss_2

In [ ]:
class Trainer(nn.Module):

    def __init__(self, net, X, hg, optimizer):
        super().__init__()
        self.X: torch.Tensor = X.to(DEVICE)
        self.hg = hg.to(DEVICE)
        self.de = self.hg.H.to_dense().sum(dim=0).to("cpu").to(DEVICE)
        self.optimizer: torch.optim.Optimizer = optimizer
        self.layers = nn.ModuleList()
        self.layers.append(net.to(DEVICE))
        self.weight = 200
    def forward(self, X):
        X = self.layers[0](X, self.hg)
        for layer in self.layers[1:]:
            X = layer(X)
        return X

    def run(self, epoch):
        self.train()  
        self.optimizer.zero_grad()
        outs = self.forward(self.X)
        loss, loss_1, loss_2 = loss_bs_matrix(outs, self.hg, device=DEVICE,weight=self.weight)
        loss.backward()
        self.optimizer.step()

        return loss.item(), loss_1.item(), loss_2.item()

In [27]:
import hgp.utils
G = hgp.utils.from_pickle_to_hypergraph("../data/cora")
edges, _ = G.e
G.num_e,G.num_v

(1413, 1330)

In [28]:
X = torch.randn(size=(G.num_v, hyper["init_features_dim"]))
# X = torch.eye(hyper["init_features_dim"])
net = HGNNP(hyper["h_hyper_prmts"]).to(DEVICE)
hgnn_trainer = Trainer(net=net, X=X, hg=G, optimizer=None).to(DEVICE)
for (k,v) in hyper["l_hyper_prmts"].items():
    hgnn_trainer.layers.append(nn.BatchNorm1d(num_features=v["in_channels"]).to(DEVICE)) if v["use_bn"] else None
    hgnn_trainer.layers.append(nn.ReLU().to(DEVICE))
    if v["drop_rate"] > 0:
        hgnn_trainer.layers.append(nn.Dropout(v["drop_rate"]))
    hgnn_trainer.layers.append(nn.Linear(in_features=v["in_channels"],out_features=v["out_channels"],device=DEVICE))
hgnn_trainer.layers.append(nn.Softmax(dim=1))

ModuleList(
  (0): HGNNP(
    (layers): ModuleList(
      (0): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.2, inplace=False)
        (theta): Linear(in_features=1330, out_features=1024, bias=True)
      )
      (1): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.1, inplace=False)
        (theta): Linear(in_features=1024, out_features=512, bias=True)
      )
      (2): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.1, inplace=False)
        (theta): Linear(in_features=512, out_features=256, bias=True)
      )
      (3): HGNNPConv(
        (bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.1, inplace=False)
        (theta): Linear(in_features=256, out_features=128, bias=True)
      )
    )
  )
  (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0

In [29]:
# hgnn_trainer.layers
# for n,p in hgnn_trainer.named_parameters():
#     print(n,p)
hgnn_trainer.weight = weight

In [30]:
temp_loss_total,temp_loss1,temp_loss2 = torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False)
optim1 = optim.Adam(hgnn_trainer.parameters(), lr=2e-5, weight_decay=5e-8)
hgnn_trainer.optimizer = optim1
for epoch in range(20000):
    if hgnn_trainer.weight > 6:
        hgnn_trainer.weight = hgnn_trainer.weight - 0.00016
    loss,loss_1,loss_2 = hgnn_trainer.run(epoch=epoch)
    temp_loss_total += loss
    temp_loss1 += loss_1
    temp_loss2 += loss_2
    if epoch % 10 == 0:
        print(f"in {epoch} epoch, average loss: {temp_loss_total.item() / 10}")
        print(f"                , loss1: {temp_loss1.item() / 10}")
        print(f"                , loss2: {temp_loss2.item() / 10}")
        print(f"                , weight: {hgnn_trainer.weight}")
        print(f"=================================")
        sys.stdout.flush()
        temp_loss_total,temp_loss1,temp_loss2 = torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False)

in 0 epoch, average loss: 636.008740234375
                , loss1: 125.1763671875
                , loss2: 10.12691879272461
                , weight: 5


in 10 epoch, average loss: 6302.96875
                , loss1: 1251.842578125
                , loss2: 43.75582580566406
                , weight: 5
in 20 epoch, average loss: 6261.37578125
                , loss1: 1251.8876953125
                , loss2: 1.9371963500976563
                , weight: 5
in 30 epoch, average loss: 6263.065234375
                , loss1: 1251.8744140625
                , loss2: 3.692513656616211
                , weight: 5
in 40 epoch, average loss: 6259.891015625
                , loss1: 1251.86259765625
                , loss2: 0.5782711029052734
                , weight: 5
in 50 epoch, average loss: 6259.75703125
                , loss1: 1251.85361328125
                , loss2: 0.4883397102355957
                , weight: 5
in 60 epoch, average loss: 6259.42578125
                , loss1: 1251.844921875
                , loss2: 0.2013988494873047
                , weight: 5
in 70 epoch, average loss: 6259.256640625
                , loss1: 1251.8051757

KeyboardInterrupt: 

In [31]:
hgnn_trainer.eval()
outs = hgnn_trainer.forward(hgnn_trainer.X)
outs_straight = StraightThroughEstimator.apply(outs)
G_clone = G.clone()
edges, _  = G_clone.e
cut = 0
for vertices in edges:
    if torch.prod(outs_straight[list(vertices)], dim=0).sum() == 0:
        cut += 1
    else:
        G_clone.remove_hyperedges(vertices)
assert cut == G_clone.num_e
cut

200

In [32]:
(abs(outs - 0.5) < 0.01).any()

tensor(False, device='cuda:0')

In [33]:
num_nodes = outs_straight.sum(dim=0)
print(num_nodes)
(torch.max(num_nodes).item() - torch.min(num_nodes).item()) / num_nodes.sum().item()


tensor([332., 332., 332., 332.], device='cuda:0', grad_fn=<SumBackward1>)


0.0

In [ ]:
torch.var(torch.tensor([53.,56,53,54,56,55]))

tensor(1.9000)